In [2]:
import json
import pandas as pd

from folium import plugins
import folium


In [8]:
descendants = pd.read_csv("data/descendants.csv")

In [9]:
descendants["id"]

0             11
1          19930
2              4
3          19939
4              1
           ...  
256764     33501
256765     74611
256766     33507
256767    235565
256768    164293
Name: id, Length: 256769, dtype: int64

In [10]:
data = pd.read_csv("data/data.csv")

In [11]:
data.head()

,id,name,thesis,school,country,year,subject,advisors,students
0,11,Alfred E. Engelbrecht,Coupled Bending and Torsional Free Vibration o...,Iowa State University,UnitedStates,1949.0,NaN,[1],[]
1,4,Charles Joseph Thorne,The Approximate Solution of Linear Differentia...,Iowa State University,UnitedStates,1941.0,NaN,[239],"[19930, 19939]"
2,1,Ernest Willard Anderson,Statics of Special Types of Homogeneous Elasti...,Iowa State University,UnitedStates,1933.0,74—Mechanics of deformable solids,[258],"[11, 28, 31]"
3,7,Carl Eric Langenhop,Properties of Kernels of Integral Equations Wh...,Iowa State University,UnitedStates,1948.0,NaN,[281],"[48, 104689, 38, 33, 3026, 39, 32, 35]"
4,6,William B. Stiles,Solutions of Clamped Plated Problems by Means ...,Iowa State University,UnitedStates,1945.0,NaN,[258],[]


In [124]:
s2 = data['id']

In [125]:
coords = pd.read_csv("data/coordinates.csv")

In [126]:
merged = pd.merge(data,coords,on='school',how='left')
merged.shape


(256769, 11)

In [159]:
merged2 = pd.merge(merged,descend,on='id',how="left")
merged2.columns

Index(['id', 'name', 'thesis', 'school', 'country', 'year', 'subject',
       'advisors', 'students', 'lat', 'lon', 'descendants'],
      dtype='object')

## Influential universities

In [146]:
inf_uni = merged2[["school","lat","lon","descendants"]].groupby(['school'])
inf_uni = inf_uni["descendants"].agg(['sum']).reset_index()
inf_uni = inf_uni.iloc[1:,:]
inf_uni.shape


(3911, 2)

In [145]:
merged3 = pd.merge(inf_uni,coords,on='school',how='left')
merged3.shape

(3911, 4)

In [147]:
merged3.head()

,school,sum,lat,lon
0,(Candidate of Sciences) Novosibirsk State Univ...,4,54.846317,83.093750
1,(Candidate of Sciences) Sobolev Institute of M...,1,55.049999,82.949997
2,(Doctorat 3e cycle) École Normale Supérie,1,48.821259,2.315226
3,(Doctorat d'Etat) Université Paris Diderot - P...,1,48.829723,2.380833
4,(Doctorat d'Etat) École Polytechnique,1,48.713268,2.210165


In [154]:
X = merged3[["lat","lon","sum"]]
X.dropna(inplace=True)
X

C:\Users\karth\AppData\Local\Temp\ipykernel_15520\1309021871.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.dropna(inplace=True)


,lat,lon,sum
0,54.846317,83.093750,4
1,55.049999,82.949997,1
2,48.821259,2.315226,1
3,48.829723,2.380833,1
4,48.713268,2.210165,1
...,...,...,...
3906,46.672527,0.366398,2
3907,48.045746,-1.746071,2
3908,59.253992,15.248768,1
3909,41.105046,29.026327,12


## Plotting

In [173]:
t0=1950
t1=1960

period = merged2[ (merged2["year"]>=t0) & (merged2['year']<t1)]

p = period[["school","lat","lon","descendants"]].groupby(['school'])
p = p["descendants"].agg(['sum']).reset_index()
p = pd.merge(p,coords,on='school',how='left')

X = p[["lat","lon","sum"]]
X.dropna(inplace=True)


C:\Users\karth\AppData\Local\Temp\ipykernel_15520\54324571.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.dropna(inplace=True)


In [174]:
map = folium.Map(location = [5,30], tiles = "Cartodb dark_matter", zoom_start=2)

heat_data = X.values.tolist()

plugins.HeatMap(heat_data).add_to(map)

map